In [1]:
import sys
import os
import pandas as pd
import numpy as np
sys.path.append('/nfs/research1/zerbino/jhidalgo/inteql/inteql/')
from modelFunctions import *
from scipy.stats import ttest_ind
from scipy import stats
import random

In [2]:
###### Load RNA-seq data #######

In [3]:
os.chdir('../inteql/data/original-data/RNA-seq/')

In [4]:
data=pd.read_csv('Top_genes_ML.csv')

In [5]:
data=data.rename(columns={'0':'Tissue'})

In [6]:
data=data.set_index('Tissue')

In [7]:
reindex_data=data.reset_index()

In [38]:
data

,ENST00000538644.1,ENST00000575363.1,ENST00000475226.1,ENST00000426995.2,ENST00000398824.2,ENST00000256104.4,ENST00000237014.3,ENST00000594907.1,ENST00000472523.1,ENST00000420959.2,...,ENST00000251595.6,ENST00000370456.4,ENST00000493888.1,ENST00000376335.3,ENST00000221847.5,ENST00000376673.3,ENST00000524255.1,ENST00000494569.1,ENST00000354822.5,ENST00000446581.1
Tissue,,,,,,,,,,,,,,,,,,,,,
Adipose - Subcutaneous,0.00,0.00,5.67,0.38,0.04,1100.00,0.68,0.35,17.95,0.00,...,289.50,0.00,3.01,0.05,1.19,0.71,0.28,0.33,0.00,0.0
Muscle - Skeletal,0.30,0.00,1.27,389.50,0.02,115.10,0.62,0.00,9.65,0.00,...,154.60,0.16,0.89,0.00,0.22,0.00,0.00,0.03,0.00,0.0
Artery - Tibial,0.00,0.00,10.13,0.36,0.00,129.20,1.50,0.00,0.14,0.00,...,170.20,0.25,0.54,0.02,0.50,0.44,0.19,0.00,0.00,0.0
Artery - Coronary,0.19,0.78,63.44,0.74,0.00,3925.00,0.82,0.00,0.00,0.00,...,4689.00,0.00,0.29,0.02,1.04,0.00,0.00,0.00,0.03,0.0
Heart - Atrial Appendage,0.00,0.00,36.72,0.30,0.15,181.20,0.64,0.00,0.00,0.00,...,1356.00,0.05,0.05,0.03,0.95,0.00,0.57,0.00,0.00,469.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ovary,1.95,0.00,1.21,0.27,0.07,1.98,0.85,0.00,0.00,0.24,...,5.90,0.09,0.00,0.00,0.71,0.70,0.37,0.00,0.03,0.0
Vagina,1.64,0.00,0.00,0.00,1.76,1.29,0.93,0.00,0.00,0.00,...,10.07,77.56,0.81,0.02,0.62,0.00,0.37,0.00,0.00,0.0
Artery - Tibial,0.00,0.00,15.13,0.35,0.03,11.35,1.01,0.00,0.10,0.24,...,447.50,0.16,0.00,0.03,3.19,0.00,0.35,0.00,0.00,0.0


In [8]:
####### Prediction functions ######

In [38]:
def t_test (i):
    results_sample_p_val={}
    results_tissue_p_val={}
    results_count={}
    binary={}
    for Transc in data.columns:
        results_sample_p_val[Transc]={}
        results_tissue_p_val[Transc]={}
        results_count[Transc]={}
        binary[Transc]={}
        for Tissue in data.index.unique():
            ttest_sample=stats.ttest_1samp(data[Transc][Tissue],data.iloc[i][Transc])#######
            ttest_tissue=ttest_ind(data[Transc][Tissue],data[Transc][data.index != Tissue])
            results_sample_p_val[Transc][Tissue]=ttest_sample.pvalue
            results_tissue_p_val[Transc][Tissue]=ttest_tissue.pvalue
            binary[Transc][Tissue]=np.nan
            if ttest_sample.pvalue >= 0.0001 and ttest_tissue.pvalue <= 0.00001: binary[Transc][Tissue]= 1## 
            if ttest_sample.pvalue <= 0.0001 and ttest_tissue.pvalue <= 0.00001: binary[Transc][Tissue]= 0##
            if ttest_tissue.pvalue <= 0.00001: 
                results_count[Transc][Tissue]= 1
            else:
                results_count[Transc][Tissue]= 0
                
    return {'tissue':results_tissue_p_val,'transcript':results_sample_p_val,'binary':binary,'count':results_count}

In [10]:
def t_test_ratio (i):
    value={}
    for Transc in data.columns:
        value[Transc]={}
        for Tissue in data.index.unique():
            ttest_sample=stats.ttest_1samp(data[Transc][Tissue],data.iloc[i][Transc])
            value[Transc][Tissue]=np.nan
            if ttest_sample.pvalue >= 0.0001: value[Transc][Tissue] = ratiodata[Transc][Tissue]## 
    return {'value':value}

In [8]:
def t_test_ratio_external (i):
    value={}
    for Transc in data.columns:
        value[Transc]={}
        for Tissue in data.index.unique():
            ttest_sample=stats.ttest_1samp(data[Transc][Tissue],i[Transc])
            value[Transc][Tissue]=np.nan
            if ttest_sample.pvalue >= 0.0001: value[Transc][Tissue] = ratiodata[Transc][Tissue]## 
    return {'value':value}

In [16]:
##### LOAD eQTLs #######

In [17]:
eqtl_file = '/nfs/research1/zerbino/jhidalgo/inteql/data/output/output05.csv.gz'

In [18]:
eQTLs=pd.read_csv(eqtl_file,sep=',',compression='gzip')

In [19]:
eQTLs

,variant_id,gene_id,Adipose_Subcutaneous,Adipose_Visceral_Omentum,Adrenal_Gland,Artery_Aorta,Artery_Coronary,Artery_Tibial,Brain_Amygdala,Brain_Anterior_cingulate_cortex_BA24,...,Skin_Sun_Exposed_Lower_leg,Small_Intestine_Terminal_Ileum,Spleen,Stomach,Testis,Thyroid,Uterus,Vagina,Whole_Blood,Cells_EBV-transformed_lymphocytes
0,19_24221728_G_C_b37,ENSG00000213096,0.263925,NaN,NaN,0.263636,NaN,0.203748,NaN,NaN,...,0.386554,NaN,NaN,0.249301,NaN,NaN,NaN,NaN,NaN,NaN
1,17_79670273_T_C_b37,ENSG00000182446,-0.105845,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-0.231742,-0.194333,NaN,NaN,NaN,NaN
2,1_151056305_C_T_b37,ENSG00000197622,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2_179580434_A_G_b37,ENSG00000223960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.717794,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5_61605063_C_A_b37,ENSG00000068796,NaN,NaN,NaN,0.254959,NaN,0.154943,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5407341,4_68898861_T_G_b37,ENSG00000198092,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.142926,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5407342,1_155415181_G_A_b37,ENSG00000203761,-0.228117,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.226778,NaN,NaN,NaN,NaN,-0.337073,NaN,NaN,NaN,NaN
5407343,4_77413179_G_A_b37,ENSG00000163749,NaN,NaN,NaN,-0.356020,NaN,-0.243742,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-0.295155,NaN,NaN,NaN,NaN
5407344,6_89399126_G_A_b37,ENSG00000111880,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
######### DICT with common names from eQTL and RNA-Seq data names #########

In [25]:
tissues_common={}
for b in data.index.unique():
    if b.replace(' - ','_').replace(' ','_').replace('(','').replace(')','') in (eQTLs.columns[2:]):
        #print(b.replace(' - ','_').replace(' ','_').replace('(','').replace(')',''),'matches.')
        tissues_common[b]=b.replace(' - ','_').replace(' ','_').replace('(','').replace(')','')
    else:
        print(b.replace(' - ','_').replace(' ','_').replace('(','').replace(')',''),'doesnt match!!!!!!!!!!!!!!')

Kidney_Cortex doesnt match!!!!!!!!!!!!!!
Fallopian_Tube doesnt match!!!!!!!!!!!!!!
Bladder doesnt match!!!!!!!!!!!!!!
Cervix_Ectocervix doesnt match!!!!!!!!!!!!!!
Cervix_Endocervix doesnt match!!!!!!!!!!!!!!


In [26]:
tissues_common_inv = {v: k for k, v in tissues_common.items()}

In [27]:
tissues_common

{'Adipose - Subcutaneous': 'Adipose_Subcutaneous',
 'Muscle - Skeletal': 'Muscle_Skeletal',
 'Artery - Tibial': 'Artery_Tibial',
 'Artery - Coronary': 'Artery_Coronary',
 'Heart - Atrial Appendage': 'Heart_Atrial_Appendage',
 'Adipose - Visceral (Omentum)': 'Adipose_Visceral_Omentum',
 'Ovary': 'Ovary',
 'Uterus': 'Uterus',
 'Vagina': 'Vagina',
 'Breast - Mammary Tissue': 'Breast_Mammary_Tissue',
 'Skin - Not Sun Exposed (Suprapubic)': 'Skin_Not_Sun_Exposed_Suprapubic',
 'Minor Salivary Gland': 'Minor_Salivary_Gland',
 'Brain - Cortex': 'Brain_Cortex',
 'Adrenal Gland': 'Adrenal_Gland',
 'Thyroid': 'Thyroid',
 'Lung': 'Lung',
 'Spleen': 'Spleen',
 'Pancreas': 'Pancreas',
 'Esophagus - Muscularis': 'Esophagus_Muscularis',
 'Esophagus - Mucosa': 'Esophagus_Mucosa',
 'Esophagus - Gastroesophageal Junction': 'Esophagus_Gastroesophageal_Junction',
 'Stomach': 'Stomach',
 'Colon - Sigmoid': 'Colon_Sigmoid',
 'Small Intestine - Terminal Ileum': 'Small_Intestine_Terminal_Ileum',
 'Colon - Tran

In [ ]:
###### Identify significant (DE) transcripts for each tissue #######

In [12]:
significant={}

In [13]:
for Tissue in data.index.unique(): significant[Tissue]=[]

In [14]:
for Transc in data.columns:
    for Tissue in data.index.unique():
        ttest=ttest_ind(data[Transc][Tissue],data[Transc][data.index != Tissue])
        if ttest.pvalue <= 0.00001:
            significant[Tissue].append(Transc)
            #print(Transc,Tissue,ttest)

In [15]:
pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in significant.items() ]))

,Adipose - Subcutaneous,Muscle - Skeletal,Artery - Tibial,Artery - Coronary,Heart - Atrial Appendage,Adipose - Visceral (Omentum),Ovary,Uterus,Vagina,Breast - Mammary Tissue,...,Brain - Caudate (basal ganglia),Brain - Nucleus accumbens (basal ganglia),Brain - Putamen (basal ganglia),Brain - Hypothalamus,Brain - Spinal cord (cervical c-1),Brain - Amygdala,Fallopian Tube,Bladder,Cervix - Ectocervix,Cervix - Endocervix
0,ENST00000538644.1,ENST00000538644.1,ENST00000538644.1,ENST00000178638.3,ENST00000538644.1,ENST00000256104.4,ENST00000178638.3,ENST00000372216.4,ENST00000178638.3,ENST00000256104.4,...,ENST00000538644.1,ENST00000538644.1,ENST00000178638.3,ENST00000178638.3,ENST00000178638.3,ENST00000178638.3,ENST00000553456.3,ENST00000325888.8,ENST00000006015.3,ENST00000006015.3
1,ENST00000256104.4,ENST00000575363.1,ENST00000256104.4,ENST00000491109.1,ENST00000472523.1,ENST00000472523.1,ENST00000498730.1,ENST00000419268.1,ENST00000498730.1,ENST00000472523.1,...,ENST00000178638.3,ENST00000498730.1,ENST00000498730.1,ENST00000498730.1,ENST00000498730.1,ENST00000498730.1,NaN,ENST00000225538.3,ENST00000559638.1,ENST00000296280.6
2,ENST00000472523.1,ENST00000426995.2,ENST00000594907.1,ENST00000297785.3,ENST00000419268.1,ENST00000372216.4,ENST00000491109.1,ENST00000498730.1,ENST00000546916.1,ENST00000498730.1,...,ENST00000498730.1,ENST00000491109.1,ENST00000491109.1,ENST00000491109.1,ENST00000491109.1,ENST00000491109.1,NaN,NaN,ENST00000296280.6,ENST00000553456.3
3,ENST00000284770.5,ENST00000256104.4,ENST00000472523.1,ENST00000448933.1,ENST00000178638.3,ENST00000178638.3,ENST00000297785.3,ENST00000491109.1,ENST00000449873.1,ENST00000491109.1,...,ENST00000491109.1,ENST00000274063.4,ENST00000274063.4,ENST00000297785.3,ENST00000546916.1,ENST00000297785.3,NaN,NaN,ENST00000553456.3,ENST00000249499.6
4,ENST00000178638.3,ENST00000372216.4,ENST00000419268.1,ENST00000375771.1,ENST00000498730.1,ENST00000274063.4,ENST00000343713.4,ENST00000449873.1,ENST00000497741.1,ENST00000274063.4,...,ENST00000274063.4,ENST00000297785.3,ENST00000546916.1,ENST00000546916.1,ENST00000558353.1,ENST00000546916.1,NaN,NaN,ENST00000249499.6,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,NaN,ENST00000390003.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
122,NaN,ENST00000438221.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123,NaN,ENST00000251595.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124,NaN,ENST00000376673.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
####### Calculate ratio of success for each transcript for each tissue #######

In [39]:
fails={}
success={}
ratio={}

In [42]:
len_tran=len(data.columns)
for Tissue in data.index.unique().sort_values():
    print(Tissue)
    fails[Tissue]=[]
    success[Tissue]=[]
    ratio[Tissue]={}
    for Transc in data.columns:
        print_cum=('Transcript '+str(data.columns.get_loc(Transc)+1)+' of '+str(len_tran)+": ")  
        ttest_tissue=ttest_ind(data[Transc][Tissue],data[Transc][data.index != Tissue])
        if ttest_tissue.pvalue <= 0.00001:
            print_cum=print_cum+'Tissue expression significant:'
            for sample in list((reindex_data[reindex_data['Tissue'] == Tissue]).index):
                print_cum_fin=print_cum+'Sample '+str((reindex_data[reindex_data['Tissue'] == Tissue]).index.get_loc(sample))+' of '+str(len(list((reindex_data[reindex_data['Tissue'] == Tissue]).index)))
                print(print_cum_fin,end="\r")
                ttest_sample=stats.ttest_1samp(data[Transc][Tissue],data.iloc[sample][Transc])
                if ttest_sample.pvalue <= 0.0001: success[Tissue].append(Transc)
                elif ttest_sample.pvalue >= 0.0001: fails[Tissue].append(Transc)
                if (data[Transc][Tissue]).mean() == 0 and data.iloc[sample][Transc] == 0:
                    success[Tissue].append(Transc)
                if (data[Transc][Tissue]).mean() == 0 and data.iloc[sample][Transc] != 0:
                    fails[Tissue].append(Transc)
            ratio[Tissue][Transc]=success[Tissue].count(Transc)/(success[Tissue].count(Transc)+fails[Tissue].count(Transc))
        else: ratio[Tissue][Transc]= np.nan
    print("")
    print('###########')

Adipose - Subcutaneous


KeyboardInterrupt: 

In [44]:
data

,ENST00000538644.1,ENST00000575363.1,ENST00000475226.1,ENST00000426995.2,ENST00000398824.2,ENST00000256104.4,ENST00000237014.3,ENST00000594907.1,ENST00000472523.1,ENST00000420959.2,...,ENST00000251595.6,ENST00000370456.4,ENST00000493888.1,ENST00000376335.3,ENST00000221847.5,ENST00000376673.3,ENST00000524255.1,ENST00000494569.1,ENST00000354822.5,ENST00000446581.1
Tissue,,,,,,,,,,,,,,,,,,,,,
Adipose - Subcutaneous,0.00,0.00,5.67,0.38,0.04,1100.00,0.68,0.35,17.95,0.00,...,289.50,0.00,3.01,0.05,1.19,0.71,0.28,0.33,0.00,0.0
Muscle - Skeletal,0.30,0.00,1.27,389.50,0.02,115.10,0.62,0.00,9.65,0.00,...,154.60,0.16,0.89,0.00,0.22,0.00,0.00,0.03,0.00,0.0
Artery - Tibial,0.00,0.00,10.13,0.36,0.00,129.20,1.50,0.00,0.14,0.00,...,170.20,0.25,0.54,0.02,0.50,0.44,0.19,0.00,0.00,0.0
Artery - Coronary,0.19,0.78,63.44,0.74,0.00,3925.00,0.82,0.00,0.00,0.00,...,4689.00,0.00,0.29,0.02,1.04,0.00,0.00,0.00,0.03,0.0
Heart - Atrial Appendage,0.00,0.00,36.72,0.30,0.15,181.20,0.64,0.00,0.00,0.00,...,1356.00,0.05,0.05,0.03,0.95,0.00,0.57,0.00,0.00,469.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ovary,1.95,0.00,1.21,0.27,0.07,1.98,0.85,0.00,0.00,0.24,...,5.90,0.09,0.00,0.00,0.71,0.70,0.37,0.00,0.03,0.0
Vagina,1.64,0.00,0.00,0.00,1.76,1.29,0.93,0.00,0.00,0.00,...,10.07,77.56,0.81,0.02,0.62,0.00,0.37,0.00,0.00,0.0
Artery - Tibial,0.00,0.00,15.13,0.35,0.03,11.35,1.01,0.00,0.10,0.24,...,447.50,0.16,0.00,0.03,3.19,0.00,0.35,0.00,0.00,0.0


In [45]:
ratiodata=pd.DataFrame(ratio).T

In [46]:
ratiodata.to_csv('Ratiodata_mod.csv',index=True)

In [10]:
ratiodata=pd.read_csv('Ratiodata_mod.csv',index_col=0)

In [13]:
for i in ratiodata.columns:
    if np.isnan(max(ratiodata[i])):
        print(i,'expression is not significant to any tissue.')
    else:
        print(i,'Max:',max(ratiodata[i]),'Min:',min(ratiodata[i]))

ENST00000538644.1 Max: 0.9148936170212766 Min: 0.4506437768240344
ENST00000575363.1 expression is not significant to any tissue.
ENST00000475226.1 expression is not significant to any tissue.
ENST00000426995.2 expression is not significant to any tissue.
ENST00000398824.2 expression is not significant to any tissue.
ENST00000256104.4 Max: 0.8665158371040724 Min: 0.22604422604422605
ENST00000237014.3 expression is not significant to any tissue.
ENST00000594907.1 expression is not significant to any tissue.
ENST00000472523.1 Max: 1.0 Min: 0.0594059405940594
ENST00000420959.2 expression is not significant to any tissue.
ENST00000372216.4 expression is not significant to any tissue.
ENST00000390315.2 expression is not significant to any tissue.
ENST00000419268.1 expression is not significant to any tissue.
ENST00000468980.2 expression is not significant to any tissue.
ENST00000284770.5 Max: 0.8599290780141844 Min: 0.627906976744186
ENST00000178638.3 Max: 0.8868613138686131 Min: 0.158249158

In [11]:
for i in ratiodata.columns:
    for b in ratiodata[i].index:
        if ratiodata[i][b]<=0.7:ratiodata[i][b]=np.nan

In [60]:
ratiodata

,ENST00000538644.1,ENST00000575363.1,ENST00000475226.1,ENST00000426995.2,ENST00000398824.2,ENST00000256104.4,ENST00000237014.3,ENST00000594907.1,ENST00000472523.1,ENST00000420959.2,...,ENST00000251595.6,ENST00000370456.4,ENST00000493888.1,ENST00000376335.3,ENST00000221847.5,ENST00000376673.3,ENST00000524255.1,ENST00000494569.1,ENST00000354822.5,ENST00000446581.1
Adipose - Subcutaneous,0.864253,NaN,NaN,NaN,NaN,0.866516,NaN,NaN,0.830317,NaN,...,NaN,NaN,NaN,NaN,NaN,0.995475,NaN,NaN,0.873303,NaN
Adipose - Visceral (Omentum),NaN,NaN,NaN,NaN,NaN,0.825352,NaN,NaN,1.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,0.997183,NaN,NaN,0.887324,NaN
Adrenal Gland,0.842105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.994737,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Artery - Aorta,0.876254,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,0.816054,NaN,...,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN
Artery - Coronary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN
Artery - Tibial,0.897959,NaN,NaN,NaN,NaN,0.809524,NaN,0.888889,0.834467,NaN,...,NaN,NaN,NaN,NaN,NaN,0.997732,NaN,NaN,0.859410,NaN
Bladder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Brain - Amygdala,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN
Brain - Anterior cingulate cortex (BA24),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.743802,NaN,NaN,NaN,NaN
Brain - Caudate (basal ganglia),0.793750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.831250,NaN,NaN,NaN,NaN


In [12]:
ratiodata.count(axis=1)

Adipose - Subcutaneous                        69
Adipose - Visceral (Omentum)                  59
Adrenal Gland                                 41
Artery - Aorta                                52
Artery - Coronary                             26
Artery - Tibial                               77
Bladder                                        0
Brain - Amygdala                              15
Brain - Anterior cingulate cortex (BA24)      17
Brain - Caudate (basal ganglia)               38
Brain - Cerebellar Hemisphere                 30
Brain - Cerebellum                            41
Brain - Cortex                                26
Brain - Frontal Cortex (BA9)                  31
Brain - Hippocampus                           24
Brain - Hypothalamus                          18
Brain - Nucleus accumbens (basal ganglia)     32
Brain - Putamen (basal ganglia)               28
Brain - Spinal cord (cervical c-1)             4
Brain - Substantia nigra                      10
Breast - Mammary Tis

In [11]:
ratiodata/ratiodata.sum()

,ENST00000538644.1,ENST00000575363.1,ENST00000475226.1,ENST00000426995.2,ENST00000398824.2,ENST00000256104.4,ENST00000237014.3,ENST00000594907.1,ENST00000472523.1,ENST00000420959.2,...,ENST00000251595.6,ENST00000370456.4,ENST00000493888.1,ENST00000376335.3,ENST00000221847.5,ENST00000376673.3,ENST00000524255.1,ENST00000494569.1,ENST00000354822.5,ENST00000446581.1
Adipose - Subcutaneous,0.040108,NaN,NaN,NaN,NaN,0.087717,NaN,NaN,0.052670,NaN,...,NaN,NaN,NaN,NaN,NaN,0.036927,NaN,NaN,0.071472,NaN
Adipose - Visceral (Omentum),NaN,NaN,NaN,NaN,NaN,0.083550,NaN,NaN,0.063433,NaN,...,NaN,NaN,NaN,NaN,NaN,0.036991,NaN,NaN,0.072620,NaN
Adrenal Gland,0.039080,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.063100,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Artery - Aorta,0.040665,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.051765,NaN,...,NaN,NaN,NaN,NaN,NaN,0.037095,NaN,NaN,NaN,NaN
Artery - Coronary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.037095,NaN,NaN,NaN,NaN
Artery - Tibial,0.041672,NaN,NaN,NaN,NaN,0.081948,NaN,0.493436,0.052933,NaN,...,NaN,NaN,NaN,NaN,NaN,0.037011,NaN,NaN,0.070335,NaN
Bladder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Brain - Amygdala,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Brain - Anterior cingulate cortex (BA24),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.027591,NaN,NaN,NaN,NaN
Brain - Caudate (basal ganglia),0.036836,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.030835,NaN,NaN,NaN,NaN


In [28]:
###### Perform predictions on random RNA-seq samples and check performance per tissue #######

In [29]:
idontknowhowtocalthisval={}

In [30]:
for Tissue in data.index.unique():
    idontknowhowtocalthisval[Tissue]=[]

In [31]:
a=eQTLs[tissues_common.values()].rename(columns=tissues_common_inv)
mean_ratio=[]
pred_ratio=[]
slope=[]
weighted_predicted=[]
mean_predicted=[]
tissue=[]
#for sample in random.sample(range(1, len(data.index)), 20000):
for sample in range(1, len(data.index)):
    if data.iloc[sample].name in tissues_common:
        if ~ np.isnan(eQTLs[tissues_common[data.iloc[sample].name]].iloc[sample]):
            print(data.iloc[sample].name)
            test=t_test_ratio_external(data.iloc[sample])
            b=pd.DataFrame(test['value']).sum(axis=1)[tissues_common.keys()]
            ### b_where_a = scores for tissues with eQTL value for such transcript
            b_where_a=b.multiply(~ np.isnan(a))
            b_where_a_sum=b_where_a.sum(axis=1)
            prediction=a.multiply(b_where_a.div(b_where_a_sum,axis=0)).iloc[sample].sum()
            real=eQTLs[tissues_common[data.iloc[sample].name]].iloc[sample]
            mean_prediction=eQTLs[eQTLs.columns[2:].drop(tissues_common[data.iloc[sample].name])].iloc[sample].mean()
            if np.isnan(mean_prediction): continue 
            print('Real/Mean:',real/mean_prediction,'Real/Predicted:',real/prediction)
            mean_ratio.append(real/mean_prediction)
            pred_ratio.append(real/prediction)
            slope.append(real)
            weighted_predicted.append(prediction)
            mean_predicted.append(mean_prediction)
            tissue.append(data.iloc[sample].name)
            print('###############')

Brain - Cortex
Real/Mean: 0.7408342602066686 Real/Predicted: 0.6984077949151042
###############
Muscle - Skeletal
Real/Mean: 1.1741791287594892 Real/Predicted: 1.370493445409404
###############
Muscle - Skeletal
Real/Mean: 0.7308453320269428 Real/Predicted: 0.8509291063265583
###############
Heart - Atrial Appendage
Lung
Real/Mean: 0.8839710400393936 Real/Predicted: 0.9651311126360183
###############
Skin - Not Sun Exposed (Suprapubic)
Real/Mean: 0.47468476468243487 Real/Predicted: 0.5694156654504325
###############
Thyroid
Real/Mean: 1.6327599237577568 Real/Predicted: 1.4505200920190133
###############
Heart - Atrial Appendage
Real/Mean: 1.0753826453198019 Real/Predicted: 1.1065233388723048
###############
Adipose - Visceral (Omentum)
Real/Mean: 0.7565292210135042 Real/Predicted: 0.8206082936677529
###############
Cells - EBV-transformed lymphocytes
Real/Mean: 0.9870094472560694 Real/Predicted: 0.9794654918968293
###############
Cells - Transformed fibroblasts
Real/Mean: 0.57352781337

Real/Mean: 0.32136334835839175 Real/Predicted: 0.3832133842605251
###############
Cells - Transformed fibroblasts
Real/Mean: 0.9258386652715535 Real/Predicted: 0.9841624315669133
###############
Artery - Aorta
Real/Mean: 0.624214427203053 Real/Predicted: 0.7748135602402244
###############
Esophagus - Muscularis
Real/Mean: 1.6376787271053963 Real/Predicted: 1.6196778054556653
###############
Colon - Transverse
Real/Mean: 1.0207917865521254 Real/Predicted: 1.014319086326236
###############
Skin - Sun Exposed (Lower leg)
Real/Mean: 0.6428897131064698 Real/Predicted: 0.8261512044592543
###############
Whole Blood
Real/Mean: -1.2152653300216265 Real/Predicted: 2.6835815588250407
###############
Whole Blood
Real/Mean: -0.6539088438182724 Real/Predicted: 1.0826199848776856
###############
Cells - Transformed fibroblasts
Skin - Not Sun Exposed (Suprapubic)
Real/Mean: 0.433658738402493 Real/Predicted: 0.6324201770541711
###############
Testis
Muscle - Skeletal
Thyroid
Real/Mean: 0.4719122101651

Real/Mean: 2.29034780473304 Real/Predicted: 1.4831044278986418
###############
Testis
Real/Mean: -0.8660787017021048 Real/Predicted: 28.37400962057824
###############
Artery - Tibial
Real/Mean: 0.6780362256231418 Real/Predicted: 0.7664378952537829
###############
Brain - Cerebellar Hemisphere
Ovary
Real/Mean: 1.0172437162574122 Real/Predicted: 1.0387659308367359
###############
Adrenal Gland
Real/Mean: 1.212670910470687 Real/Predicted: 1.1473919145331175
###############
Lung
Real/Mean: 0.7658658960162269 Real/Predicted: 0.81847179594621
###############
Adipose - Visceral (Omentum)
Real/Mean: 0.5186708313717413 Real/Predicted: 0.660905821599618
###############
Stomach
Real/Mean: 1.045819740771556 Real/Predicted: 1.0944045676820504
###############
Artery - Tibial
Real/Mean: 0.8898133737800056 Real/Predicted: 0.9169025597531301
###############
Cells - Transformed fibroblasts
Real/Mean: 0.6421466875617873 Real/Predicted: 0.8317703270351671
###############
Pancreas
Real/Mean: 1.108698917157

Real/Mean: 0.792211244722825 Real/Predicted: 0.8942855766466202
###############
Nerve - Tibial
Real/Mean: 0.5566561312608663 Real/Predicted: 0.7468992572844694
###############
Artery - Tibial
Real/Mean: 0.7188158639039617 Real/Predicted: 0.7653623577546876
###############
Thyroid
Real/Mean: 0.7844206497588614 Real/Predicted: 0.838286035989423
###############
Esophagus - Gastroesophageal Junction
Real/Mean: 0.9550621527457905 Real/Predicted: 0.9696175406275783
###############
Skin - Not Sun Exposed (Suprapubic)
Real/Mean: 0.6764485943404712 Real/Predicted: 0.7164583270589118
###############
Artery - Tibial
Real/Mean: 0.5956044460751657 Real/Predicted: 0.7770495490782922
###############
Nerve - Tibial
Real/Mean: 0.37471801142239036 Real/Predicted: 0.5898826422798232
###############
Brain - Hippocampus
Real/Mean: 1.295247709947231 Real/Predicted: 1.2859448297752225
###############
Heart - Atrial Appendage
Real/Mean: 1.267069896072153 Real/Predicted: 1.3252044206084495
###############
Esop

Real/Mean: 1.0692441571357534 Real/Predicted: 1.0517290501821792
###############
Esophagus - Muscularis
Skin - Sun Exposed (Lower leg)
Real/Mean: 0.9259500092264579 Real/Predicted: 0.9792896672282301
###############
Muscle - Skeletal
Real/Mean: 0.8361765808822061 Real/Predicted: 0.9456046290620768
###############
Thyroid
Real/Mean: 0.7086320153721567 Real/Predicted: 0.8426712369794602
###############
Adipose - Subcutaneous
Real/Mean: 0.6798534936497573 Real/Predicted: 0.7058803256574855
###############
Muscle - Skeletal
Real/Mean: 1.3519472029269746 Real/Predicted: 1.1353996233851904
###############
Nerve - Tibial
Adipose - Visceral (Omentum)
Real/Mean: 1.0568816672226136 Real/Predicted: 1.0313284365482798
###############
Lung
Real/Mean: 0.7260537310193201 Real/Predicted: 0.8951581178425673
###############
Thyroid
Real/Mean: 0.9973422206714833 Real/Predicted: 0.9714183410191292
###############
Skin - Sun Exposed (Lower leg)
Real/Mean: 0.9756365135229399 Real/Predicted: 1.02031234840631

...

Heart - Atrial Appendage
Real/Mean: 0.6793993472456334 Real/Predicted: 0.7184682227637355
###############
Cells - Transformed fibroblasts
Real/Mean: 0.48264883066166925 Real/Predicted: 0.5072354432174434
###############
Cells - Transformed fibroblasts
Real/Mean: 2.168402142761236 Real/Predicted: 1.3489311192141107
###############
Thyroid
Esophagus - Mucosa
Real/Mean: 0.6213985223355231 Real/Predicted: 0.8041533283048026
###############
Colon - Transverse
Real/Mean: 1.1400809544325476 Real/Predicted: 1.1208352628589409
###############
Muscle - Skeletal
Real/Mean: 1.233758009887818 Real/Predicted: 1.0992340685352613
###############
Colon - Sigmoid
Real/Mean: 0.8861208833423405 Real/Predicted: 0.9281406792684299
###############
Lung
Real/Mean: 1.0364713284348641 Real/Predicted: 1.0362278287441173
###############
Esophagus - Mucosa
Real/Mean: 0.8644245151203661 Real/Predicted: 0.8913224221134712
###############
Testis
Real/Mean: 1.0484192232385185 Real/Predicted: 1.012209019859594
########

Real/Mean: 1.2951788886069524 Real/Predicted: 1.0800224982034394
###############
Whole Blood
Artery - Tibial
Real/Mean: 0.7429133694694996 Real/Predicted: 0.9392795251505472
###############
Cells - Transformed fibroblasts
Real/Mean: 1.123950000175512 Real/Predicted: 1.1243210980174088
###############
Cells - Transformed fibroblasts
Real/Mean: 0.8152697924233459 Real/Predicted: 0.895075859658887
###############
Testis
Real/Mean: 1.6267292036481604 Real/Predicted: 1.132516215187544
###############
Adipose - Visceral (Omentum)
Real/Mean: 0.7359477117153356 Real/Predicted: 0.7741797033775902
###############
Nerve - Tibial
Real/Mean: 3.030222381261229 Real/Predicted: 1.5256171399914207
###############
Whole Blood
Real/Mean: 0.5516843724057122 Real/Predicted: 0.5887045795860303
###############
Skin - Not Sun Exposed (Suprapubic)
Real/Mean: 0.8276081803501593 Real/Predicted: 1.015392336339572
###############
Esophagus - Mucosa
Real/Mean: 1.0645509479076654 Real/Predicted: 0.9963774800492274
#

Real/Mean: 0.6964923276164225 Real/Predicted: 0.7517263677602682
###############
Esophagus - Mucosa
Real/Mean: 0.8572909310044289 Real/Predicted: 0.8514263304810402
###############
Artery - Tibial
Real/Mean: 1.1807090097805875 Real/Predicted: 1.1189964906794154
###############
Heart - Left Ventricle
Real/Mean: 1.1331363678009079 Real/Predicted: 1.1470291909042434
###############
Breast - Mammary Tissue
Real/Mean: 1.1936305195973704 Real/Predicted: 1.1485074879635095
###############
Skin - Sun Exposed (Lower leg)
Real/Mean: 1.0041238463049027 Real/Predicted: 1.0012327216167594
###############
Artery - Tibial
Real/Mean: 1.2986349693353079 Real/Predicted: 1.1856951055959557
###############
Whole Blood
Real/Mean: 2.9428231160630203 Real/Predicted: 1.4444748789161383
###############
Artery - Aorta
Real/Mean: 0.9518320516884673 Real/Predicted: 1.062861674616475
###############
Thyroid
Real/Mean: 1.64460063378565 Real/Predicted: 1.5552450294512992
###############
Muscle - Skeletal
Real/Mean: 

Real/Mean: 0.8146528412261391 Real/Predicted: 0.9454421663730217
###############
Lung
Real/Mean: 0.8868246925977821 Real/Predicted: 0.9647694107849194
###############
Muscle - Skeletal
Real/Mean: 0.992986277806564 Real/Predicted: 1.025848005237155
###############
Artery - Aorta
Real/Mean: 1.1862071453860326 Real/Predicted: 1.1785223238994478
###############
Heart - Left Ventricle
Real/Mean: 0.6346343920565148 Real/Predicted: 0.7152212206395266
###############
Nerve - Tibial
Thyroid
Real/Mean: 0.6554469558229712 Real/Predicted: 0.7600785432463403
###############
Heart - Left Ventricle
Real/Mean: 1.0991126481576583 Real/Predicted: 0.9605390335551294
###############
Esophagus - Mucosa
Colon - Sigmoid
Real/Mean: 2.0020569280281113 Real/Predicted: 1.677165089078555
###############
Skin - Not Sun Exposed (Suprapubic)
Real/Mean: 1.1021946498955835 Real/Predicted: 1.1824724576943015
###############
Cells - Transformed fibroblasts
Real/Mean: 0.4893735483576497 Real/Predicted: 0.5073913272291193

Real/Mean: 3.064377103977548 Real/Predicted: 1.5874428794576356
###############
Skin - Sun Exposed (Lower leg)
Real/Mean: 0.898695306358453 Real/Predicted: 0.9496859578130498
###############
Thyroid
Real/Mean: -0.6404492414125271 Real/Predicted: 1.2145371237802027
###############
Artery - Tibial
Real/Mean: 0.8554264800135476 Real/Predicted: 0.8560522045699684
###############
Colon - Sigmoid
Real/Mean: 0.7585974309575997 Real/Predicted: 0.7675139281995368
###############
Adipose - Subcutaneous
Real/Mean: 0.9983244105485117 Real/Predicted: 0.970561634627437
###############
Muscle - Skeletal
Real/Mean: 0.699211961725882 Real/Predicted: 0.7521489842284361
###############
Nerve - Tibial
Real/Mean: 0.5692694936134864 Real/Predicted: 0.68499313500232
###############
Artery - Tibial
Lung
Real/Mean: 1.325471536535849 Real/Predicted: 1.2506404069530466
###############
Artery - Aorta
Real/Mean: 1.0420456957488136 Real/Predicted: 0.9973381591330918
###############
Colon - Transverse
Real/Mean: 1.0

In [32]:
Results=pd.DataFrame({'Tissue':tissue,'Slope':slope,'Predicted':weighted_predicted,'Predicted Ratio':pred_ratio,'Mean':mean_predicted,'Mean Ratio':mean_ratio})

In [36]:
Results.to_csv('Results0907.csv',index=False)

In [ ]:
####### Random tests from dataset ######

In [178]:
for sample in random.sample(range(1, len(data.index)), 100):
    test=t_test_ratio(sample)
    print(data.iloc[sample].name)
    print(pd.DataFrame(test['value']).sum(axis=1).sort_values(ascending=False)[:5])
    print('###############')

Heart - Atrial Appendage
Heart - Atrial Appendage    4.191919
Liver                       3.514286
Lung                        2.550351
Stomach                     2.423664
Brain - Cortex              2.316456
dtype: float64
###############
Whole Blood
Whole Blood                        9.862408
Brain - Hypothalamus               3.661157
Pancreas                           3.330645
Brain - Caudate (basal ganglia)    2.993750
Heart - Left Ventricle             2.950495
dtype: float64
###############
Adipose - Subcutaneous
Adipose - Subcutaneous     5.882353
Lung                       4.170960
Breast - Mammary Tissue    3.365517
Ovary                      2.909774
Nerve - Tibial             2.541063
dtype: float64
###############
Small Intestine - Terminal Ileum
Stomach                             2.667939
Esophagus - Mucosa                  2.496314
Small Intestine - Terminal Ileum    2.233577
Cells - Transformed fibroblasts     1.755102
Lung                                1.714286
dtyp

Prostate
Colon - Transverse    4.189781
Esophagus - Mucosa    4.159705
Lung                  3.964871
Prostate              3.572368
Colon - Sigmoid       2.287554
dtype: float64
###############
Colon - Sigmoid
Breast - Mammary Tissue    5.658621
Lung                       4.131148
Nerve - Tibial             4.062802
Artery - Tibial            3.541950
Thyroid                    3.434978
dtype: float64
###############
Adrenal Gland
Heart - Atrial Appendage    4.309764
Muscle - Skeletal           3.796099
Brain - Cortex              3.727848
Stomach                     3.400763
Whole Blood                 3.395577
dtype: float64
###############
Esophagus - Muscularis
Artery - Aorta                  3.986622
Artery - Tibial                 3.421769
Thyroid                         3.394619
Adipose - Visceral (Omentum)    3.309859
Testis                          3.235521
dtype: float64
###############
Muscle - Skeletal
Muscle - Skeletal                           9.808511
Stomach           

Adipose - Subcutaneous
Artery - Tibial                 4.219955
Lung                            4.084309
Esophagus - Mucosa              2.594595
Adipose - Visceral (Omentum)    2.538028
Brain - Hippocampus             2.178862
dtype: float64
###############
Liver
Liver                                        7.617143
Brain - Anterior cingulate cortex (BA24)     3.826446
Heart - Left Ventricle                       3.676568
Brain - Putamen (basal ganglia)              3.661290
Brain - Nucleus accumbens (basal ganglia)    3.537415
dtype: float64
###############
Liver
Muscle - Skeletal                  5.828014
Cells - Transformed fibroblasts    5.827988
Liver                              3.777143
Pancreas                           3.201613
Whole Blood                        2.584767
dtype: float64
###############
Brain - Hippocampus
Brain - Nucleus accumbens (basal ganglia)    4.095238
Brain - Cortex                               3.778481
Liver                                        3.76

Spleen
Cells - Transformed fibroblasts    4.072886
Adrenal Gland                      3.847368
Brain - Hypothalamus               3.644628
Brain - Caudate (basal ganglia)    3.612500
Brain - Cortex                     3.575949
dtype: float64
###############
Vagina
Stomach                                4.988550
Colon - Transverse                     4.259124
Skin - Sun Exposed (Lower leg)         3.310782
Skin - Not Sun Exposed (Suprapubic)    3.266150
Cells - Transformed fibroblasts        2.559767
dtype: float64
###############
Artery - Coronary
Artery - Tibial           4.224490
Uterus                    4.009009
Muscle - Skeletal         3.964539
Esophagus - Muscularis    3.310811
Lung                      3.262295
dtype: float64
###############
Brain - Putamen (basal ganglia)
Brain - Cerebellum                           5.687861
Whole Blood                                  3.933661
Heart - Left Ventricle                       3.867987
Brain - Nucleus accumbens (basal ganglia)    3

In [62]:
ratiotest=t_test_ratio(55)

In [63]:
print(data.iloc[55].name)

Adipose - Subcutaneous


In [64]:
pd.DataFrame(ratiotest['value']).sum(axis=1).sort_values(ascending=False)[:5]

Adipose - Subcutaneous             5.929864
Breast - Mammary Tissue            5.034483
Adipose - Visceral (Omentum)       3.957746
Skin - Sun Exposed (Lower leg)     3.350951
Cells - Transformed fibroblasts    3.309038
dtype: float64

In [ ]:
###### Prediction on the mean RNA-seq value per tissue #######

In [13]:
tissue_means=data.groupby('Tissue').mean()

In [64]:
tissue_means

,ENST00000538644.1,ENST00000575363.1,ENST00000475226.1,ENST00000426995.2,ENST00000398824.2,ENST00000256104.4,ENST00000237014.3,ENST00000594907.1,ENST00000472523.1,ENST00000420959.2,...,ENST00000251595.6,ENST00000370456.4,ENST00000493888.1,ENST00000376335.3,ENST00000221847.5,ENST00000376673.3,ENST00000524255.1,ENST00000494569.1,ENST00000354822.5,ENST00000446581.1
Tissue,,,,,,,,,,,,,,,,,,,,,
Adipose - Subcutaneous,0.140475,0.379163,13.724502,0.342308,0.057534,2204.174457,0.664661,0.101516,3.909819,0.057715,...,437.784412,0.045023,1.440905,0.066878,2.596538,0.137081,0.269412,0.135294,0.015204,0.101946
Adipose - Visceral (Omentum),0.719408,0.103437,29.817521,0.251915,0.074873,2461.177746,0.994282,0.076563,0.010648,0.046873,...,1339.768620,0.045211,0.969718,0.044028,3.691944,0.113352,0.328817,0.001042,0.018563,0.096197
Adrenal Gland,0.183053,0.087368,4.438158,0.263474,0.040316,9.542368,0.263158,0.054211,0.030316,0.035316,...,147.176684,0.045737,0.075474,0.018053,3.396895,1.000789,0.196158,0.001632,0.015105,0.049895
Artery - Aorta,0.353378,0.114983,23.837291,0.261639,0.097425,28.215953,0.304749,0.076990,0.335151,0.083846,...,1113.113679,0.050167,0.142341,0.033411,3.691070,0.032642,0.494582,0.010669,0.026722,0.464281
Artery - Coronary,0.337803,1.470925,20.011792,0.312197,0.098150,247.223699,0.576994,0.086069,0.114740,0.079017,...,982.764798,0.062370,0.382139,0.020462,3.627168,0.081040,1.308728,0.006532,0.013699,1.094624
Artery - Tibial,0.222449,0.119206,20.879909,0.498730,0.050000,37.457370,0.522268,0.236054,0.198231,0.128957,...,626.151383,0.047256,0.138254,0.041633,1.502177,0.055102,0.302381,0.009819,0.022132,0.146553
Bladder,1.184545,0.450000,1.327273,0.353636,0.833636,88.707273,0.460000,0.030909,0.010909,0.183636,...,58.418182,0.299091,0.798182,0.018182,2.153636,0.500909,0.008182,0.000000,0.000000,0.093636
Brain - Amygdala,0.220000,0.195300,13.363300,0.118500,0.039400,0.570300,31.129400,0.055900,0.969200,0.030800,...,355.806500,0.043600,0.033900,3.380800,1.719800,1.997000,0.993600,0.082400,0.194000,0.106400
Brain - Anterior cingulate cortex (BA24),0.267769,0.141074,11.073058,0.147851,0.059835,0.522397,10.071488,0.019421,2.674959,0.057769,...,284.235372,0.050992,0.027190,5.748760,1.114628,3.763058,1.094793,0.214711,0.097107,0.181901


In [206]:
for i in tissue_means.index:
    print(i)
    test=t_test_ratio_external(pred_data.loc[i])
    print(pd.DataFrame(test['value']).sum(axis=1).sort_values(ascending=False)[:5])
    print('############################')
    

Adipose - Subcutaneous
Adipose - Subcutaneous            42.020362
Adipose - Visceral (Omentum)       6.833803
Nerve - Tibial                     5.212560
Breast - Mammary Tissue            5.106897
Skin - Sun Exposed (Lower leg)     4.325581
dtype: float64
############################
Adipose - Visceral (Omentum)
Adipose - Visceral (Omentum)    36.557746
Adipose - Subcutaneous           7.018100
Lung                             6.039813
Nerve - Tibial                   4.519324
Breast - Mammary Tissue          4.134483
dtype: float64
############################
Adrenal Gland
Adrenal Gland               26.894737
Stomach                      6.931298
Muscle - Skeletal            5.998227
Heart - Atrial Appendage     5.552189
Pancreas                     4.076613
dtype: float64
############################
Artery - Aorta
Artery - Aorta                  39.799331
Adipose - Visceral (Omentum)     4.081690
Esophagus - Muscularis           3.516216
Whole Blood                      2.805897

Esophagus - Gastroesophageal Junction    16.315574
Esophagus - Muscularis                    7.216216
Adipose - Subcutaneous                    4.160633
Heart - Left Ventricle                    4.026403
Adipose - Visceral (Omentum)              3.425352
dtype: float64
############################
Esophagus - Mucosa
Esophagus - Mucosa                     49.088452
Stomach                                 4.935115
Skin - Not Sun Exposed (Suprapubic)     4.426357
Lung                                    3.742389
Muscle - Skeletal                       3.689716
dtype: float64
############################
Esophagus - Muscularis
Esophagus - Muscularis                   28.708108
Artery - Tibial                           7.092971
Esophagus - Gastroesophageal Junction     6.983607
Lung                                      3.981265
Colon - Sigmoid                           3.866953
dtype: float64
############################
Fallopian Tube
Stomach                                4.171756
Prostate

In [ ]:
####### tests with predictions ####### (Can be deleted?)

In [268]:
pd.DataFrame(test['value']).sum(axis=1).sort_values(ascending=False)[tissues_common.keys()]/pd.DataFrame(test['value']).sum(axis=1).sort_values(ascending=False)[tissues_common.keys()].sum()

Adipose - Subcutaneous                       0.007273
Muscle - Skeletal                            0.030550
Artery - Tibial                              0.013861
Artery - Coronary                            0.000000
Heart - Atrial Appendage                     0.006165
Adipose - Visceral (Omentum)                 0.013768
Ovary                                        0.004606
Uterus                                       0.000000
Vagina                                       0.000000
Breast - Mammary Tissue                      0.000475
Skin - Not Sun Exposed (Suprapubic)          0.006884
Minor Salivary Gland                         0.000000
Brain - Cortex                               0.010761
Adrenal Gland                                0.021992
Thyroid                                      0.011344
Lung                                         0.000742
Spleen                                       0.001020
Pancreas                                     0.019486
Esophagus - Muscularis      

In [21]:
#eQTLs[tissues_common.values()].rename(columns=tissues_common_inv)

In [ ]:
#prediction=(eQTLs[tissues_common.values()].rename(columns=tissues_common_inv)*pd.DataFrame(test['value']).sum(axis=1).sort_values(ascending=False)[tissues_common.keys()]/pd.DataFrame(test['value']).sum(axis=1).sort_values(ascending=False)[tissues_common.keys()].sum()).sum(axis=1)

In [ ]:
#real=eQTLs[tissues_common[i]]

In [ ]:
#mean_prediction=eQTLs.mean(axis=1)

In [37]:
eQTLs[eQTLs.columns[2:].drop(i)].mean(axis=1)

0    0.283726
1   -0.177307
2    0.136124
3    0.727376
4    0.169599
dtype: float64

In [21]:
eQTLs[tissues_common.values()].rename(columns=tissues_common_inv)

,Adipose - Subcutaneous,Muscle - Skeletal,Artery - Tibial,Artery - Coronary,Heart - Atrial Appendage,Adipose - Visceral (Omentum),Ovary,Uterus,Vagina,Breast - Mammary Tissue,...,Brain - Substantia nigra,Brain - Anterior cingulate cortex (BA24),Brain - Frontal Cortex (BA9),Brain - Cerebellar Hemisphere,Brain - Caudate (basal ganglia),Brain - Nucleus accumbens (basal ganglia),Brain - Putamen (basal ganglia),Brain - Hypothalamus,Brain - Spinal cord (cervical c-1),Brain - Amygdala
0,0.263925,0.378815,0.203748,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.105845,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,0.136124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,0.154943,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5407341,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5407342,-0.228117,-0.213302,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5407343,NaN,NaN,-0.243742,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5407344,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [142]:
pd.DataFrame(test['value']).sum(axis=1)[tissues_common.keys()]

Adipose - Subcutaneous                        9.058824
Muscle - Skeletal                             2.053191
Artery - Tibial                               7.324263
Artery - Coronary                             0.000000
Heart - Atrial Appendage                      1.814815
Adipose - Visceral (Omentum)                  5.202817
Ovary                                         1.714286
Uterus                                        2.648649
Vagina                                       16.669565
Breast - Mammary Tissue                       5.086207
Skin - Not Sun Exposed (Suprapubic)           6.082687
Minor Salivary Gland                          3.793814
Brain - Cortex                                1.360759
Adrenal Gland                                 2.689474
Thyroid                                       5.901345
Lung                                          9.613583
Spleen                                        2.302469
Pancreas                                      2.754032
Esophagus 

In [143]:
pd.DataFrame(test['value']).sum(axis=1)[tissues_common.keys()].sum()

168.54763871526112

In [25]:
a=eQTLs[tissues_common.values()].rename(columns=tissues_common_inv)

In [26]:
b=pd.DataFrame(test['value']).sum(axis=1)[tissues_common.keys()]

In [27]:
c=pd.DataFrame(test['value']).sum(axis=1)[tissues_common.keys()].sum()

In [28]:
prediction=(a*(b/c)).sum(axis=1)

In [84]:
pred={}
for transc in a.index:
    bb=b[~ np.isnan(a.loc[transc])]
    cc=bb.sum()
    pred[transc]=a.loc[transc]*bb/cc

KeyboardInterrupt: 

In [123]:
what=a.multiply(b_where_a.div(b_where_a_sum,axis=0)).sum(axis=1)

In [ ]:
####### Prediction and eQTL comparison for each mean RNA-seq value ######

In [18]:
prediction={}
real={}
mean_prediction={}
### a = renamed eQTLs
a=eQTLs[tissues_common.values()].rename(columns=tissues_common_inv)

for i in tissues_common.keys():
    print(i)
    test=t_test_ratio_external(tissue_means.loc[i])
    ### b = tissue scores according to T test
    b=pd.DataFrame(test['value']).sum(axis=1)[tissues_common.keys()]
    ### b_where_a = scores for tissues with eQTL value for such transcript
    b_where_a=b.multiply(~ np.isnan(a))
    b_where_a_sum=b_where_a.sum(axis=1)
    prediction[i]=a.multiply(b_where_a.div(b_where_a_sum,axis=0)).sum(axis=1)
    #prediction[i]=(eQTLs[tissues_common.values()].rename(columns=tissues_common_inv)*pd.DataFrame(test['value']).sum(axis=1)[tissues_common.keys()]/pd.DataFrame(test['value']).sum(axis=1)[tissues_common.keys()].sum()).sum(axis=1)
    real[i]=eQTLs[tissues_common[i]]
    mean_prediction[i]=eQTLs[eQTLs.columns[2:].drop(tissues_common[i])].mean(axis=1)
    #print(pd.DataFrame(test['value']).sum(axis=1).sort_values(ascending=False)[:5])
    print('############################')

Adipose - Subcutaneous
############################
Muscle - Skeletal
############################
Artery - Tibial
############################
Artery - Coronary
############################
Heart - Atrial Appendage
############################
Adipose - Visceral (Omentum)
############################
Ovary
############################
Uterus
############################
Vagina
############################
Breast - Mammary Tissue
############################
Skin - Not Sun Exposed (Suprapubic)
############################
Minor Salivary Gland
############################
Brain - Cortex
############################
Adrenal Gland
############################
Thyroid
############################
Lung
############################
Spleen
############################
Pancreas
############################
Esophagus - Muscularis
############################
Esophagus - Mucosa
############################
Esophagus - Gastroesophageal Junction
############################
Stomach
###############

In [51]:
Predicted=pd.DataFrame(prediction)

ValueError: DataFrame constructor not properly called!

In [52]:
Real=pd.DataFrame(real)

ValueError: DataFrame constructor not properly called!

In [21]:
Mean_predicted=pd.DataFrame(mean_prediction)

In [53]:
Predicted

NameError: name 'Predicted' is not defined

In [53]:
Mean_predicted

,Adipose - Subcutaneous,Muscle - Skeletal,Artery - Tibial,Artery - Coronary,Heart - Atrial Appendage,Adipose - Visceral (Omentum),Ovary,Uterus,Vagina,Breast - Mammary Tissue,...,Brain - Substantia nigra,Brain - Anterior cingulate cortex (BA24),Brain - Frontal Cortex (BA9),Brain - Cerebellar Hemisphere,Brain - Caudate (basal ganglia),Brain - Nucleus accumbens (basal ganglia),Brain - Putamen (basal ganglia),Brain - Hypothalamus,Brain - Spinal cord (cervical c-1),Brain - Amygdala
0,0.285706,0.274217,0.291724,0.283726,0.283726,0.283726,0.283726,0.283726,0.283726,0.283726,...,0.283726,0.283726,0.283726,0.283726,0.283726,0.283726,0.283726,0.283726,0.283726,0.283726
1,-0.213037,-0.177307,-0.177307,-0.177307,-0.177307,-0.177307,-0.177307,-0.177307,-0.177307,-0.177307,...,-0.177307,-0.177307,-0.177307,-0.177307,-0.177307,-0.177307,-0.177307,-0.177307,-0.177307,-0.177307
2,0.136124,NaN,0.136124,0.136124,0.136124,0.136124,0.136124,0.136124,0.136124,0.136124,...,0.136124,0.136124,0.136124,0.136124,0.136124,0.136124,0.136124,0.136124,0.136124,0.136124
3,0.727376,0.727376,0.727376,0.727376,0.727376,0.727376,0.727376,0.727376,0.727376,0.727376,...,0.727376,0.727376,0.727376,0.727376,0.727376,0.727376,0.727376,0.727376,0.727376,0.727376
4,0.169599,0.169599,0.174484,0.169599,0.169599,0.169599,0.169599,0.169599,0.169599,0.169599,...,0.169599,0.169599,0.169599,0.169599,0.169599,0.169599,0.169599,0.169599,0.169599,0.169599
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5407341,-0.154019,-0.154019,-0.154019,-0.154019,-0.154019,-0.154019,-0.154019,-0.154019,-0.154019,-0.154019,...,-0.154019,-0.154019,-0.154019,-0.154019,-0.154019,-0.154019,-0.154019,-0.154019,-0.154019,-0.154019
5407342,-0.305262,-0.306744,-0.298249,-0.298249,-0.298249,-0.298249,-0.298249,-0.298249,-0.298249,-0.298249,...,-0.298249,-0.298249,-0.298249,-0.298249,-0.298249,-0.298249,-0.298249,-0.298249,-0.298249,-0.298249
5407343,-0.290932,-0.290932,-0.298797,-0.290932,-0.290932,-0.290932,-0.290932,-0.290932,-0.290932,-0.290932,...,-0.290932,-0.290932,-0.290932,-0.290932,-0.290932,-0.290932,-0.290932,-0.290932,-0.290932,-0.290932
5407344,-0.269984,-0.269984,-0.269984,-0.269984,-0.269984,-0.269984,-0.269984,-0.269984,-0.269984,-0.269984,...,-0.269984,-0.269984,-0.269984,-0.269984,-0.269984,-0.269984,-0.269984,-0.269984,-0.269984,-0.269984


In [22]:
Real_mean_corr={}
Real_pred_corr={}
for i in Real.columns:
    Real_mean_corr[i]=Real[i].corr(Mean_predicted[i])
    Real_pred_corr[i]=Real[i].corr(Predicted[i])

In [24]:
#pd.DataFrame.from_dict(Real_mean_corr,orient='index')

In [25]:
Results=pd.DataFrame([Real_mean_corr,Real_pred_corr],index=['Real-Mean corr','Real-Pred corr']).T

In [26]:
Results['Difference']=Results['Real-Pred corr']-Results['Real-Mean corr']

In [27]:
Results['Difference'].mean()

0.04410521317367224

In [45]:
Results

,Tissue,Slope,Predicted,Predicted Ratio,Mean,Mean Ratio
0,Brain - Cortex,0.359411,0.514615,0.698408,0.485144,0.740834
1,Muscle - Skeletal,-0.529525,-0.386375,1.370493,-0.450975,1.174179
2,Muscle - Skeletal,-0.328658,-0.386234,0.850929,-0.449696,0.730845
3,Lung,0.247732,0.256682,0.965131,0.280249,0.883971
4,Skin - Not Sun Exposed (Suprapubic),0.154758,0.271784,0.569416,0.326023,0.474685
...,...,...,...,...,...,...
1465,Nerve - Tibial,-0.324707,-0.383713,0.846224,-0.400151,0.811461
1466,Lung,0.163302,0.193560,0.843678,0.215052,0.759359
1467,Adipose - Visceral (Omentum),-0.269327,-0.302135,0.891413,-0.296264,0.909078
1468,Skin - Not Sun Exposed (Suprapubic),-0.712777,-0.644467,1.105995,-0.626857,1.137065


In [50]:
Results.merge(pd.DataFrame(ratiodata.count(axis=1),columns=['Sig Transc']),how='left',left_index=True,right_index=True).merge(pd.DataFrame((ratiodata/ratiodata.sum()).mean(axis=1),columns=['Mean sig transc score']),how='left',left_index=True,right_index=True)#.to_csv('Results_to_plot.csv',index=True)

,Tissue,Slope,Predicted,Predicted Ratio,Mean,Mean Ratio,Sig Transc,Mean sig transc score
0,Brain - Cortex,0.359411,0.514615,0.698408,0.485144,0.740834,NaN,NaN
1,Muscle - Skeletal,-0.529525,-0.386375,1.370493,-0.450975,1.174179,NaN,NaN
2,Muscle - Skeletal,-0.328658,-0.386234,0.850929,-0.449696,0.730845,NaN,NaN
3,Lung,0.247732,0.256682,0.965131,0.280249,0.883971,NaN,NaN
4,Skin - Not Sun Exposed (Suprapubic),0.154758,0.271784,0.569416,0.326023,0.474685,NaN,NaN
...,...,...,...,...,...,...,...,...
1465,Nerve - Tibial,-0.324707,-0.383713,0.846224,-0.400151,0.811461,NaN,NaN
1466,Lung,0.163302,0.193560,0.843678,0.215052,0.759359,NaN,NaN
1467,Adipose - Visceral (Omentum),-0.269327,-0.302135,0.891413,-0.296264,0.909078,NaN,NaN
1468,Skin - Not Sun Exposed (Suprapubic),-0.712777,-0.644467,1.105995,-0.626857,1.137065,NaN,NaN


In [49]:
pd.DataFrame(ratiodata.count(axis=1),columns=['Sig Transc'])

,Sig Transc
Adipose - Subcutaneous,69
Adipose - Visceral (Omentum),59
Adrenal Gland,41
Artery - Aorta,52
Artery - Coronary,26
Artery - Tibial,77
Bladder,0
Brain - Amygdala,15
Brain - Anterior cingulate cortex (BA24),17
Brain - Caudate (basal ganglia),38


In [35]:
Results['Difference'].mean()

0.04410521317367224

In [48]:
ratiodata

,ENST00000538644.1,ENST00000575363.1,ENST00000475226.1,ENST00000426995.2,ENST00000398824.2,ENST00000256104.4,ENST00000237014.3,ENST00000594907.1,ENST00000472523.1,ENST00000420959.2,...,ENST00000251595.6,ENST00000370456.4,ENST00000493888.1,ENST00000376335.3,ENST00000221847.5,ENST00000376673.3,ENST00000524255.1,ENST00000494569.1,ENST00000354822.5,ENST00000446581.1
Adipose - Subcutaneous,0.864253,NaN,NaN,NaN,NaN,0.866516,NaN,NaN,0.830317,NaN,...,NaN,NaN,NaN,NaN,NaN,0.995475,NaN,NaN,0.873303,NaN
Adipose - Visceral (Omentum),NaN,NaN,NaN,NaN,NaN,0.825352,NaN,NaN,1.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,0.997183,NaN,NaN,0.887324,NaN
Adrenal Gland,0.842105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.994737,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Artery - Aorta,0.876254,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.816054,NaN,...,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN
Artery - Coronary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN
Artery - Tibial,0.897959,NaN,NaN,NaN,NaN,0.809524,NaN,0.888889,0.834467,NaN,...,NaN,NaN,NaN,NaN,NaN,0.997732,NaN,NaN,0.859410,NaN
Bladder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Brain - Amygdala,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Brain - Anterior cingulate cortex (BA24),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.743802,NaN,NaN,NaN,NaN
Brain - Caudate (basal ganglia),0.793750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.831250,NaN,NaN,NaN,NaN
